In [33]:
import os
import math

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [34]:
base_census = pd.read_csv('/Users/fernandovieira/OneDrive/1. Educacao/INFO E ESTAT/IA Expert/ML-e-DS-com-Python-de-A-a-Z/Bases de dados/census.csv')
base_census

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


Preparação

In [35]:
x_census = base_census.iloc[:, 0:14].values
x_census

array([[39, ' State-gov', 77516, ..., 0, 40, ' United-States'],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 13, ' United-States'],
       [38, ' Private', 215646, ..., 0, 40, ' United-States'],
       ...,
       [58, ' Private', 151910, ..., 0, 40, ' United-States'],
       [22, ' Private', 201490, ..., 0, 20, ' United-States'],
       [52, ' Self-emp-inc', 287927, ..., 0, 40, ' United-States']],
      dtype=object)

In [36]:
y_census = base_census.iloc[:, 14].values
y_census

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)

In [37]:
# Tratar atributos categóricos
from sklearn.preprocessing import LabelEncoder

In [38]:
label_encoder_workclass = LabelEncoder()
label_encoder_education = LabelEncoder()
label_encoder_marital = LabelEncoder()
label_encoder_occupation = LabelEncoder()
label_encoder_relationship = LabelEncoder()
label_encoder_race = LabelEncoder()
label_encoder_sex = LabelEncoder()
label_encoder_country = LabelEncoder()

In [39]:
x_census[:, 1] = label_encoder_workclass.fit_transform(x_census[:, 1])
x_census[:, 3] = label_encoder_education.fit_transform(x_census[:, 3])
x_census[:, 5] = label_encoder_marital.fit_transform(x_census[:, 5])
x_census[:, 6] = label_encoder_occupation.fit_transform(x_census[:, 6])
x_census[:, 7] = label_encoder_relationship.fit_transform(x_census[:, 7])
x_census[:, 8] = label_encoder_race.fit_transform(x_census[:, 8])
x_census[:, 9] = label_encoder_sex.fit_transform(x_census[:, 9])
x_census[:, 13] = label_encoder_country.fit_transform(x_census[:, 13])

In [40]:
# Padronizar os dados
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [41]:
x_census = scaler.fit_transform(x_census)
x_census

array([[ 0.03067056,  2.15057856, -1.06361075, ..., -0.21665953,
        -0.03542945,  0.29156857],
       [ 0.83710898,  1.46373585, -1.008707  , ..., -0.21665953,
        -2.22215312,  0.29156857],
       [-0.04264203,  0.09005041,  0.2450785 , ..., -0.21665953,
        -0.03542945,  0.29156857],
       ...,
       [ 1.42360965,  0.09005041, -0.35877741, ..., -0.21665953,
        -0.03542945,  0.29156857],
       [-1.21564337,  0.09005041,  0.11095988, ..., -0.21665953,
        -1.65522476,  0.29156857],
       [ 0.98373415,  0.77689313,  0.92989258, ..., -0.21665953,
        -0.03542945,  0.29156857]])

In [42]:
# Dividir em treinamento e teste
from sklearn.model_selection import train_test_split

In [43]:
x_census_treinamento, x_census_teste, y_census_treinamento, y_census_teste = train_test_split(x_census, y_census, test_size=0.15, random_state=0)

In [44]:
x_census_treinamento.shape, x_census_teste.shape

((27676, 14), (4885, 14))

TÉCNICA 1: PCA

In [45]:
# Esta técnica seleciona os atributos mais correlacionados e os "combina".

In [46]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8) # o n_components é definido pelo analista

In [47]:
x_census_treinamento_pca = pca.fit_transform(x_census_treinamento) # fit_transform: treina
x_census_teste_pca = pca.transform(x_census_teste) # transform: apenas aplica

In [48]:
x_census_treinamento_pca.shape, x_census_teste_pca.shape

((27676, 8), (4885, 8))

In [49]:
x_census_treinamento

array([[-0.92239303,  1.46373585,  1.84273483, ..., -0.21665953,
        -0.03542945,  0.29156857],
       [-0.84908045,  0.09005041,  0.19630465, ..., -0.21665953,
         3.20416118,  0.29156857],
       [ 1.42360965, -1.28363502,  2.83572992, ..., -0.21665953,
        -0.03542945,  0.29156857],
       ...,
       [-1.14233079,  0.09005041, -1.36867471, ..., -0.21665953,
        -0.03542945,  0.29156857],
       [ 0.47054606, -1.28363502,  0.24712496, ..., -0.21665953,
         0.36951938,  0.29156857],
       [-0.99570562,  0.09005041, -0.02703374, ..., -0.21665953,
         0.61248868,  0.29156857]])

In [50]:
pca.explained_variance_ratio_

array([0.151561  , 0.10109701, 0.08980379, 0.08076277, 0.07627678,
       0.07357646, 0.06772289, 0.06690789])

In [51]:
pca.explained_variance_ratio_.sum()

0.707708594319935

In [52]:
# Testando com o modelo Random Forest
from sklearn.ensemble import RandomForestClassifier

In [53]:
random_forest_census_pca = RandomForestClassifier(n_estimators=40, random_state=0)
random_forest_census_pca.fit(x_census_treinamento_pca, y_census_treinamento)

RandomForestClassifier(n_estimators=40, random_state=0)

In [54]:
previsoes = random_forest_census_pca.predict(x_census_teste_pca)
previsoes

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)

In [55]:
y_census_teste

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' <=50K'],
      dtype=object)

In [56]:
from sklearn.metrics import accuracy_score

In [57]:
accuracy_score(y_census_teste, previsoes)

0.8364380757420675

TÉCNICA 2: Kernel PCA

In [58]:
# Mesma lógica do SVM

In [59]:
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(n_components=8, kernel='rbf') # o n_components é definido pelo analista. rbf: utilizado em problemas não lineares

In [60]:
x_census_treinamento_kpca = pca.fit_transform(x_census_treinamento) # fit_transform: treina
x_census_teste_kpca = pca.transform(x_census_teste) # transform: apenas aplica

In [61]:
x_census_treinamento_kpca.shape, x_census_teste_kpca.shape

((27676, 8), (4885, 8))

In [32]:
x_census_treinamento_kpca

array([[ 0.72756979, -0.63707837, -0.92959677, ...,  0.4750941 ,
         1.26767572, -0.70907366],
       [ 1.94861194,  0.52336888, -0.11546966, ...,  0.20608772,
        -0.28547192, -1.10996543],
       [ 0.90984576, -0.49631087,  0.21043617, ...,  0.22000605,
         1.54790866, -0.6082255 ],
       ...,
       [-1.55068874,  1.8706316 , -1.18829813, ..., -0.15217589,
        -1.23888631,  0.23631102],
       [ 1.50148337,  0.90027   ,  1.39911867, ...,  0.83827868,
        -0.03400375,  0.51140853],
       [ 0.39972037,  0.73809833, -0.78746921, ...,  0.39215363,
        -0.75329109, -0.00358243]])

In [62]:
random_forest_census_kpca = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state=0)
random_forest_census_kpca.fit(x_census_treinamento_kpca, y_census_treinamento)

RandomForestClassifier(criterion='entropy', n_estimators=40, random_state=0)

In [63]:
previsoes = random_forest_census_kpca.predict(x_census_teste_kpca)
previsoes

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)

In [64]:
y_census_teste

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' <=50K'],
      dtype=object)

In [65]:
from sklearn.metrics import accuracy_score

In [66]:
accuracy_score(y_census_teste, previsoes)

0.8372569089048106

TÉCNICA 3: LDA

In [ ]:
# Ao contrario dos outros dois, é um algoritmo supervisionado.
# Limitado pela qtde de classes em y.

In [84]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=1) # o n_components é definido pelo analista. rbf: utilizado em problemas não lineares

In [85]:
x_census_treinamento_lda = lda.fit_transform(x_census_treinamento, y_census_treinamento) # fit_transform: treina
x_census_teste_lda = lda.transform(x_census_teste) # transform: apenas aplica

In [86]:
x_census_treinamento_lda.shape, x_census_teste_lda.shape

((27676, 1), (4885, 1))

In [87]:
random_forest_census_lda = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state=0)
random_forest_census_lda.fit(x_census_treinamento_lda, y_census_treinamento)

RandomForestClassifier(criterion='entropy', n_estimators=40, random_state=0)

In [88]:
previsoes = random_forest_census_lda.predict(x_census_teste_lda)
previsoes

array([' <=50K', ' >50K', ' <=50K', ..., ' >50K', ' >50K', ' >50K'],
      dtype=object)

In [89]:
y_census_teste

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' <=50K'],
      dtype=object)

In [90]:
from sklearn.metrics import accuracy_score

In [91]:
accuracy_score(y_census_teste, previsoes)

0.7334698055271238